## Load, enrich, augment and clean the data

In [1]:
# dataset is taken from Kaggle and downloaded from here:
# https://www.kaggle.com/datasets/thomasnibb/amsterdam-house-price-prediction
# license is CC0: Public Domain - can copy, modify, distribute and perform the work, 
# even for commercial purposes, all without asking permission.

import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

df = pd.read_csv("HousingPrices-Amsterdam-August-2021.csv")
df.head() # look of the original dataset


,Unnamed: 0,Address,Zip,Price,Area,Room,Lon,Lat
0,1,"Blasiusstraat 8 2, Amsterdam",1091 CR,685000.0,64,3,4.907736,52.356157
1,2,"Kromme Leimuidenstraat 13 H, Amsterdam",1059 EL,475000.0,60,3,4.850476,52.348586
2,3,"Zaaiersweg 11 A, Amsterdam",1097 SM,850000.0,109,4,4.944774,52.343782
3,4,"Tenerifestraat 40, Amsterdam",1060 TH,580000.0,128,6,4.789928,52.343712
4,5,"Winterjanpad 21, Amsterdam",1036 KN,720000.0,138,5,4.902503,52.410538


In [2]:
# STEP 1 - make columns look good:
df.columns = df.columns.str.lower()

# We will use only 4 columns for the start
used_cols = [ 'zip', 'address', 'area', 'room', 'price']
df = df[used_cols]
df.head(10)

,zip,address,area,room,price
0,1091 CR,"Blasiusstraat 8 2, Amsterdam",64,3,685000.0
1,1059 EL,"Kromme Leimuidenstraat 13 H, Amsterdam",60,3,475000.0
2,1097 SM,"Zaaiersweg 11 A, Amsterdam",109,4,850000.0
3,1060 TH,"Tenerifestraat 40, Amsterdam",128,6,580000.0
4,1036 KN,"Winterjanpad 21, Amsterdam",138,5,720000.0
5,1051 AM,"De Wittenkade 134 I, Amsterdam",53,2,450000.0
6,1033 KM,"Pruimenstraat 18 B, Amsterdam",87,3,450000.0
7,1053 WL,"Da Costakade 32 II, Amsterdam",80,2,590000.0
8,1058 DG,"Postjeskade 41 2, Amsterdam",49,3,399000.0
9,1073 TM,"Van Ostadestraat 193 H, Amsterdam",33,2,300000.0


In [3]:
df.describe()

,area,room,price
count,924.000000,924.000000,9.200000e+02
mean,95.952381,3.571429,6.220654e+05
std,57.447436,1.592332,5.389942e+05
min,21.000000,1.000000,1.750000e+05
25%,60.750000,3.000000,3.500000e+05
50%,83.000000,3.000000,4.670000e+05
75%,113.000000,4.000000,7.000000e+05
max,623.000000,14.000000,5.950000e+06


In [4]:
df.zip.nunique() # 834 - too many for grouping...

834

In [5]:
df.address.nunique() # 919 - too many as well...

919

In [6]:
# idea - we need to extract city districts (wijk in Dutch) based on the zip/street address
# collect a sentiment score for each district in range 1-10 using LLM - good or bad for living
# and add it as columns to our dataset - district and sentiment - as realtors often say - location, location, location...

# I decided to drop GPS lat long columns - not to mess up with Google maps API for simplicity
# and used Amsterdam district names from here:
# https://data.europa.eu/data/datasets/22433-postcode-buurt-wijk?locale=da

# idea 2 - in Amsterdam city district - wijk consists of subdistricts - buurt
# which consists of PC4 post codes (first 4 digits of zip column in our dataset)
# I will drop the street address for now and map buurt and wijk from downloaded CSV to PC4

In [7]:
# small explanation how Dutch postal addresses are organised
# The PC4 (first four digits of zip code) identifies a broad area within a city — 
# this can be mapped to neighbourhood or district statistically.
# The PC6 (full 6 characters of zip code, including letters) often maps to one side 
# of a street, or even a cluster of house numbers.
# The letter pair (e.g., CR in zip = '1091 CR') is simply a unique subdivision to make post sorting precise.

# Example:
# full zip  = 1091 CR
# street address: Blasiusstraat 8 2, Amsterdam
# Neighbourhood (Buurt): Weesperzijde
# District (Wijk): Oud-Oost
# Part of the city (Stadsdeel): Amsterdam-Oost

# so the naming hierachy is postcode → buurt → wijk


# prepare our df zip code col for mapping
df['pc4'] = df.zip.str.split().str[0]
df['PC6'] = df.zip.str.replace(' ', '')
df.head(1)

,zip,address,area,room,price,pc4,PC6
0,1091 CR,"Blasiusstraat 8 2, Amsterdam",64,3,685000.0,1091,1091CR


In [8]:
df.pc4.nunique() # 70 - much better for grouping

70

In [9]:
# STEP 2 - download district and area codes - buurt and wijk - from national statistics bureau:
# https://www.cbs.nl/nl-nl/maatwerk/2023/35/buurt-wijk-en-gemeente-2023-voor-postcode-huisnummer

# mapping = pd.read_csv('pc6hnr20230801_gwb.csv', delimiter=';')
# mapping.head(1)

In [10]:
# merge dataframes

# df = df.merge(mapping, on='PC6')
# df

In [11]:
200

200